<a href="https://colab.research.google.com/github/RayhanNuansa/Komputasi_Intelegensia/blob/main/MidtermTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nama : Muhammad Rayhan Nuansa Adha

NPM : 2006571053

In [ ]:
!pip install torch
!pip install transformers
!pip install datasets

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Example input text
input_text = "The quick brown fox jumps over the lazy dog."

# Tokenize and encode input
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Inspect inputs
print(inputs)

{'input_ids': tensor([[  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,  3899,
          1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# Perform inference
outputs = model(**inputs)
logits = outputs.logits

# Predicted class
predicted_class = logits.argmax().item()
print(f"Predicted class: {predicted_class}")

Predicted class: 1


In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
from torch.utils.data import DataLoader, Subset

# Load the dataset (e.g., SST-2 for sentiment analysis)
dataset = load_dataset("glue", "sst2")

# Define a function to tokenize the input with a shorter max length
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=32)

# Tokenize the entire dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Create subsets of the tokenized dataset
train_subset = tokenized_dataset["train"].select(range(1000))  # First 1000 samples
val_subset = tokenized_dataset["validation"].select(range(200))  # First 200 samples

# Convert tokenized subsets to DataLoader for efficient batching
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32)

# Set up training arguments with more aggressive optimizations
training_args = TrainingArguments(
    output_dir="./results",                # Output directory for saving model and checkpoints
    per_device_train_batch_size=32,        # Larger batch size
    evaluation_strategy="epoch",           # Perform evaluation at the end of each epoch
    logging_strategy="steps",              # Log training loss every few steps
    logging_steps=10,                      # Log training loss every 10 steps
    num_train_epochs=3,                    # Number of training epochs
    fp16=True,                             # Enable mixed precision for faster training
    report_to="none"                       # Disable logging to external platforms like Weights & Biases
)
# Initialize Trainer with the subsets
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=val_subset
)

# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.427000,0.546033
2,0.246200,0.473760
3,0.085900,0.491917


TrainOutput(global_step=96, training_loss=0.28395523441334564, metrics={'train_runtime': 629.458, 'train_samples_per_second': 4.766, 'train_steps_per_second': 0.153, 'total_flos': 24837637248000.0, 'train_loss': 0.28395523441334564, 'epoch': 3.0})